# CS356 lab 4:  SQL Injection Attack

SQL injection is a code injection technique used to attack data-driven applications, in which malicious SQL statements are inserted into an entry field for execution (e.g. to dump the database contents to the attacker). SQL injection is a widely used to attack databases. In fact, the SQLi attack is one of the most prevalent and dangerous security threats.

Most web applications ask for user information such as personal information, location, credit card information and others. This information is usually transferred to and from a database. SQL queries are formed based on user input which will query the database to send and receive information for giving best possible experience to users.

THe SQLi attack is designed to send malicious SQL commands to the database server. This attack is performed for malicious purpose like to extract bulk of data, dump entire data records from database etc. 

## A Quick Glance to SQLi Attack

<ul>
    <li> First, an attacker find the vulnerability in a custom Web application.</li>
    <li> The attacker then injects an SQL command to a database by sending it to the Web server. </li>
    <li> The Web server receives the malicious code and send it to different layers of server finally reaching the database server.</li>
    <li> The database server executes the malicious code on the database. The database returns data.</li>
    <li>The data is returned to Web server which in turn is displayed to the attacker</li>
</ul>

In this way, the attacker gets hold of unauthorized data. The attacker can even construct the malicious input to perform other unauthorized operations on a database server such as removing entire data from the table, dropping a table or changing data.

## Detail on SQL and SQLi attack

SQL query normally looks like this

    SELECT * FROM users WHERE username = John

This query search in 'users' table. Within the 'users' table it search in column 'username'. It returns back all the record within table 'users' with 'username' as John.

The programmer generally create such sql query based on user input. Consider something like this

    user_name = John
    query = "SELECT * FROM users WHERE username='%s'" %user_name

In the above statement, programmer is creating a variable named query which select all the record from 'users' table when 'username' match to John which is represented by user_name.

The above query evaluates to

    SELECT * FROM users WHERE username='John'

Let's assume some attacker found vulnerability in an application and realized that the application is susceptible to SQLi attack. Then, the attacker can write the input as

    user_name = ' OR 1=1; --
    query = "SELECT * FROM users WHERE username='%s'" %user_name

Now, this query will be changed to

    SELECT * FROM users WHERE username = ' OR 1=1; --

In the above input, attacker has passed no value to the username column. But the statement "OR 1=1" has been passed which will always evaluate to true(Tautology). This implies that our entire query will evaluate to true. Therefore, it will release every records from the database.

At this point, you might be wondering what ';' and '--' are doing at an end while passing the input. The ';' tells the database that it is an end of the query. The '--' implies that anything after it will be treated as a comment or will not be executed. 

Moreover, an attacker can also create malicious input which deletes an entire record from the user's table. 

    user_name = '; DELETE FROM users; --
    query = "SELECT * FROM users WHERE username='%s'" %user_name

The above query will be changed as

    SELECT * FROM users WHERE username = '; DELETE FROM users; --

This query in turn will delete entire record from users table.

## Assignment 

We will be using a sqlite database for this assignment. SQLite is a light weight database which comes prebundled with python and it reduces the overhead of other database like 'MySQL', or 'PostgreSQL'.

Necessary imports

In [1]:
import sqlite3
from getpass import getpass

The code below connects to sqlite database 'assignment.db'. If there is no database named 'assignment.db', it will create one and connect to it.

In [2]:
conn = sqlite3.connect('assignment.db')   # make a connection to the database, create it if it doesn't exist
c = conn.cursor()                         # a cursor is a pointer to the data base to perform manipulations

We have an empty database at this point. So, we will create table named 'account' in the database. Run the below cell once to create table named 'account'. We are creating account table with three columns, id (which will serve as a Primary Key for our table), username(stores user's name) and password(which stores the user's password). 

In [3]:
c.execute('''CREATE TABLE account (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, username TEXT, password TEXT)''')

The function provided below inserts two different record in the account. Here, we are storing passwords in plaintext on accounts table. Remember, it's a horrible idea to do that. But, for this assignment we are not going to go in that detail.

In [29]:
def insert_data():
    c.execute("INSERT INTO account(username, password) VALUES('mark', 'abcdef')")
    c.execute("INSERT INTO account(username, password) VALUES('paul', 'xywz123')")
    conn.commit()
    print("Successfully entered data")

insert_data()

Successfully entered data


The function below 'show_table' will display entire record of table account.

In [5]:
def show_table():
    c.execute("SELECT * FROM account")
    print(c.fetchall())

show_table()

[(1, 'mark', 'abcdef'), (2, 'paul', 'xywz123')]


The function below 'sqli' takes the username and password and checks if it matches to the record in our table. If it is present then it will simply let us enter into the system and prints "Successful login". Whereas, it will print "Failed Login" if the record is not present. Execute the following cell to simply create the python function.

In [10]:
def sqli():
    u_name = str(input("Enter Username\t"))
    print("Enter Password:\t")
    p_word = getpass()
    try:
        c.execute("SELECT username, password FROM account WHERE username='%s' AND password='%s'" %(u_name, p_word))
    except:
        c.executescript("SELECT username, password FROM account WHERE username='%s' AND password='%s'" %(u_name, p_word))
    if c.fetchone():
        print("Successful login")
    else:
        print("Failed login")

### Assignment : Part 1

In the cell below we are calling sqli() function which will prompt you to pass username and password. First, pass the expected username and password which is printed by show_table() function above to see how it behaves. Then pass unexpected username and password to see how it behaves. Now, as a part of your assignment bypass this system to print "Successful login" without inputing the database record i.e 'mark' or 'paul' in the username prompt.

In [19]:
sqli() 

Enter Password:	
Successful login


### Assignment : Part 2

At this point, you should have successfully bypassed the login. Next, in the below cell, you will invoke the same function sqli() again.  However this time you are to delete an entire **record** from the table which is a part of your assignment. The SQL syntax for deleting a record is as follows:

`DELETE FROM tablename WHERE fieldname="value";`

If you did this correctly you will get a failed login.  That doesn't matter.
Call the 'show_table()' function to see whether you deleted the record or not. If it still returns the previously created record, keep trying again. If not, then you're all set!

In [31]:
show_table()
sqli()

[(2, 'paul', 'xywz123'), (3, 'mark', 'abcdef'), (4, 'paul', 'xywz123')]
Enter Password:	
Failed login


In [32]:
show_table()

[(3, 'mark', 'abcdef')]


<u><i> Note</i></u> : If you deleted the entire record and want to play again, you can populate the table again by calling 'insert_data()' function. After populating the table, you will notice the difference in id value. Try to figure out why database behaves this way yourself.

## Preventing SQLi

By now, we are pretty familiar with what is SQLi attack and how it is normally performed. But we need to learn more about how we can prevent such attacks. Preventing attacks always come down to good programming practice, thoughtful strategies and so much more. There is no single set of techniques one could follow which would ultimately make your software free of vulnerabilities. Security is an ongoing process. But, some counter measures that can help to prevent SQLi attack are as 

<ul>
    <li> <b> Manual defensive coding practices</b> : Applying best and defensive coding practices </li>
    <li> <b> Sanitizing User Input : </b> Checking for user input before processing it any further </li>
    <li> <b> Parameterized Query Insertion : </b> Specifying the structure of an SQL query and passing the value parameters to it seperately such that any unsanitary user input is not allowed to modify the query structure. </li>
    <li> <b> Trust No one : </b> This doesn't only apply to the end users of your application. If you are using libraries created by other developers, you shouldn't trust them too and be on a lookout for vulnerabilites that might be present in the libraries you are using. </li>
    <li> <b> Firewall : </b> Use of firewall to filter out malicious data. </li>
    <li> <b> Access Control : </b> Appropriate access control and privilege will help to minimize attacks. </li>
</ul>

### Assignment : Part 3

The cell below is similar to the original sqli function defined earlier -- but it gives you a place to add code to sanitize inputs.  Add you code, then try doing a sql injection attack attack using a tautology to see if your protection mechanism works.

In [59]:
def sqli():
    u_name = str(input("Enter Username\t"))
    print("Enter Password:\t")
    p_word = getpass()
    #sanitize u_name and p_word here. You are free to do it your way.
    u_name = u_name.strip("\'\"-")
    p_word = p_word.strip("\'\"-")

    try:
        c.execute("SELECT username, password FROM account WHERE username='%s' AND password='%s'" %(u_name, p_word))
    except:
        c.executescript("SELECT username, password FROM account WHERE username='%s' AND password='%s'" %(u_name, p_word))
    if c.fetchone():
        print("Successful login")
    else:
        print("Failed login")
        
sqli()

Enter Password:	
Failed login


### What to turn in

Create a file named solution.txt. Copy the two malicious input you created in the above procedure. Briefly describe the input. Copy the python cell containing your modifications and include this with your submission, along with its output. 